### Exemples de query possible sur DuckDB pour de l'analyses

#### Pour acceder aux données de DuckDB en local depuis le volume montée :
kubectl apply -f jupyter-pod.yaml
kubectl port-forward pod/jupyter-duckdb 8888:8888 -n airflow
#### Pour choper le token :
kubectl logs pod/jupyter-duckdb -n airflow

In [13]:
import duckdb

# Connexion au fichier DuckDB monté dans le pod
conn = duckdb.connect("analytics.duckdb")

In [14]:
# Liste des tables disponibles
conn.sql("SHOW TABLES").df()

,name
0,gold_notif_impact_per_day
1,gold_user_activity_per_day


In [15]:
# Nbr de ligne dans la table gold_notif_impact_per_day
conn.sql("SELECT COUNT(*) FROM gold_notif_impact_per_day").df()

,count_star()
0,225050


In [16]:
# Nombre de lignes dans la table gold_notif_impact_per_day
conn.sql("SELECT COUNT(*) FROM gold_user_activity_per_day").df()

,count_star()
0,89950


1) Afficher les 10 premières lignes de chaque tables :

In [17]:
conn.sql("SELECT * FROM gold_notif_impact_per_day LIMIT 10").df()

,user_id,content_notif,notif_date,is_success,time_spend_after_success
0,009c8453-e395-5e81-8ad2-a8d1587395df,abandoned sediment: Architecto veniam cernuus ...,2025-03-23,False,0 days 00:24:59.585000
1,009c8453-e395-5e81-8ad2-a8d1587395df,accurate cap: Conitor solum eligendi certe vin...,2025-03-24,True,0 days 01:46:07.967000
2,009c8453-e395-5e81-8ad2-a8d1587395df,acidic sushi: Dolore vivo sopor absque ascit e...,2025-04-13,False,0 days 01:15:28.964000
3,009c8453-e395-5e81-8ad2-a8d1587395df,adolescent hydrolyze: Animadverto similique as...,2025-03-21,False,0 days 02:57:34.228000
4,009c8453-e395-5e81-8ad2-a8d1587395df,adolescent minor: Decumbo vicissitudo confero ...,2025-03-24,False,0 days 00:00:00
5,009c8453-e395-5e81-8ad2-a8d1587395df,aggravating corporation: Sustineo textor apost...,2025-04-12,True,0 days 04:01:27.858000
6,009c8453-e395-5e81-8ad2-a8d1587395df,alive hunger: Crastinus curo excepturi fuga.,2025-04-07,True,0 days 04:55:17.457000
7,009c8453-e395-5e81-8ad2-a8d1587395df,all bungalow: Degusto demonstro vulnus defluo ...,2025-04-02,True,0 days 05:18:15.594000
8,009c8453-e395-5e81-8ad2-a8d1587395df,all giant: Abstergo umquam benevolentia.,2025-03-22,False,0 days 00:00:00
9,009c8453-e395-5e81-8ad2-a8d1587395df,anguished redesign: Via amaritudo celebrer the...,2025-04-04,True,0 days 00:00:00


In [18]:
conn.sql("SELECT * FROM gold_user_activity_per_day LIMIT 10").df()

,user_id,date,total_nbr_connexion,total_spend_time,total_message_send_this_day,total_money_spend
0,009c8453-e395-5e81-8ad2-a8d1587395df,2024-10-27,0,0 days,0,12.26
1,009c8453-e395-5e81-8ad2-a8d1587395df,2024-10-30,0,0 days,0,37.10
2,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-02,0,0 days,0,2.31
3,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-04,0,0 days,0,45.77
4,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-05,0,0 days,0,49.64
5,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-07,0,0 days,0,22.32
6,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-10,0,0 days,0,39.75
7,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-11,0,0 days,0,18.03
8,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-15,0,0 days,0,7.47
9,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-23,0,0 days,0,3.27


2) Durée moyenne de session après notification

In [19]:
conn.sql("""
    SELECT 
        notif_date, 
        ROUND(AVG(EXTRACT(EPOCH FROM time_spend_after_success))/60, 2) AS avg_time_spent_minutes
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY notif_date DESC LIMIT 10
""").df()

,notif_date,avg_time_spent_minutes
0,2025-04-22,28.02
1,2025-04-21,81.57
2,2025-04-20,139.25
3,2025-04-19,153.06
4,2025-04-18,168.72
5,2025-04-17,213.99
6,2025-04-16,210.10
7,2025-04-15,219.19
8,2025-04-14,211.42
9,2025-04-13,213.28


3) Top contenus de notifications les plus engageants

In [20]:
conn.sql("""
    SELECT 
        content_notif,
        COUNT(*) AS nb_notifs,
        ROUND(AVG(EXTRACT(EPOCH FROM time_spend_after_success))/60, 2) AS avg_time_minutes
    FROM gold_notif_impact_per_day
    GROUP BY content_notif
    ORDER BY avg_time_minutes DESC
    LIMIT 10
""").df()

,content_notif,nb_notifs,avg_time_minutes
0,short affiliate: Abduco vulgus debeo amor aetas.,1,1554.60
1,agile digit: Turba arceo valde cimentarius vul...,1,1454.10
2,ample bog: Viriliter acceptus labore vere.,1,1383.82
3,imaginative onset: Fuga stabilis centum quisqu...,1,1383.82
4,fixed fax: Vos creber averto arbitro conicio.,1,1353.16
5,aching prohibition: Vivo verbera harum tergo u...,1,1353.16
6,long plain: Solio itaque pariatur spectaculum ...,1,1353.16
7,silver aircraft: Somnus causa pel crebro una a...,1,1343.98
8,enchanting cow: Ambulo reprehenderit abstergo ...,1,1343.98
9,cooperative reboot: Tener deficio caterva tabe...,1,1333.58


4) Taux de succès de notifications (notifications vues)

In [24]:
conn.sql("""
    SELECT 
        notif_date,
        SUM(CASE WHEN is_success THEN 1 ELSE 0 END) AS seen,
        COUNT(*) AS total,
        ROUND(100.0 * SUM(CASE WHEN is_success THEN 1 ELSE 0 END) / COUNT(*), 2) AS success_rate
    FROM gold_notif_impact_per_day
    GROUP BY notif_date
    ORDER BY success_rate DESC LIMIT 10
""").df()

,notif_date,seen,total,success_rate
0,2025-03-20,843.0,2441,34.54
1,2025-04-22,631.0,1831,34.46
2,2025-03-27,2174.0,6364,34.16
3,2025-04-04,2121.0,6217,34.12
4,2025-03-29,2137.0,6274,34.06
5,2025-03-23,2100.0,6169,34.04
6,2025-04-14,3380.0,9933,34.03
7,2025-04-09,2174.0,6391,34.02
8,2025-04-08,2140.0,6294,34.00
9,2025-04-15,3406.0,10072,33.82


5) Classement des clients par montant total dépensé (du plus élevé au plus faible)

In [22]:
conn.sql("""
    SELECT 
        user_id,
        SUM(total_money_spend) AS total_spent,
        COUNT(date) AS active_days,
        ROUND(AVG(total_money_spend), 2) AS avg_spent_per_day
    FROM gold_user_activity_per_day
    GROUP BY user_id
    ORDER BY total_spent DESC
    LIMIT 20
""").df()

,user_id,total_spent,active_days,avg_spent_per_day
0,02d499ef-07db-50bf-b430-3589e4804f17,4832.46,135,35.80
1,e0ea72fb-ae2a-5955-8472-716e37b3f8e4,4084.98,117,34.91
2,65e32085-d618-5dd4-8383-403d3844b62e,4038.29,118,34.22
3,861a7a83-242f-5b53-8930-860d45057f74,3956.33,115,34.40
4,49ccfc98-062a-51eb-ae40-fa2ad5963911,3904.48,122,32.00
5,bbf019ae-487f-5d00-960e-764f6784aa0b,3884.13,121,32.10
6,254c904a-96e5-5e55-8263-50af62adc388,3873.42,123,31.49
7,d6411655-ead8-5856-b3c3-103c5e43a64b,3789.41,115,32.95
8,91412fdb-ede1-5c37-b615-e2b54caa632a,3744.42,122,30.69
9,64f14d7c-dd6f-5b04-9fe1-d70f118f639b,3702.71,127,29.16


In [23]:
conn.sql("SELECT * FROM gold_user_activity_per_day").df()

,user_id,date,total_nbr_connexion,total_spend_time,total_message_send_this_day,total_money_spend
0,009c8453-e395-5e81-8ad2-a8d1587395df,2024-10-27,0,0 days 00:00:00,0,12.26
1,009c8453-e395-5e81-8ad2-a8d1587395df,2024-10-30,0,0 days 00:00:00,0,37.10
2,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-02,0,0 days 00:00:00,0,2.31
3,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-04,0,0 days 00:00:00,0,45.77
4,009c8453-e395-5e81-8ad2-a8d1587395df,2024-11-05,0,0 days 00:00:00,0,49.64
...,...,...,...,...,...,...
89945,fff01e80-c256-5c2b-b1c6-85967359c0c9,2025-04-18,5,0 days 23:20:31.291000,0,0.00
89946,fff01e80-c256-5c2b-b1c6-85967359c0c9,2025-04-19,4,0 days 13:51:54.592000,0,0.00
89947,fff01e80-c256-5c2b-b1c6-85967359c0c9,2025-04-20,3,0 days 13:07:53.478000,0,12.93
89948,fff01e80-c256-5c2b-b1c6-85967359c0c9,2025-04-21,2,0 days 18:03:45.453000,0,24.63
